Муравьиные алгоритмы


In [1]:
# import pandas as pd

# df = pd.DataFrame(tsp.cities_to_dict().items(), columns=['name', 'coords'])
# df[['x', 'y']] = pd.DataFrame(df.coords.tolist(), index=df.index)
# df.drop(columns=['coords'], inplace=True)
# df

Сверху были тестовые данные. Теперь мы уже умеем работать с классом TSP

In [2]:
import pandas as pd

cities_df = pd.read_csv('tests/16cities_1.txt', sep=' ', header=None)
cities_df.columns = ['name', 'x', 'y']

cities_df

,name,x,y
0,A,93,48
1,B,67,72
2,C,66,44
3,D,99,65
4,E,18,90
5,F,47,17
6,G,84,8
7,H,16,67
8,I,9,57
9,J,76,28


In [3]:
from algorithms.ant import AntColony
from tsp import TSP
from typing import List

In [4]:
cities: List[TSP.City] = []
for _, row in cities_df.iterrows():
    id, x, y = row['name'], row['x'], row['y']
    cities.append(TSP.City(id, int(x), int(y)))

tsp = TSP(cities)
initial_state = TSP.State(1 << 0, 0)

as_settings = AntColony.Settings()
as_colony = AntColony(AntColony.Variation.ANT_SYSTEM, as_settings)

In [5]:
%%time
path, dist = as_colony.solve(initial_state, tsp.successors, tsp.goal, tsp.add_generation)
print("Rank-based Ant System: ", [p.current_node for p in path], dist)

Rank-based Ant System:  [0, 3, 1, 10, 2, 13, 14, 8, 15, 7, 4, 11, 9, 12, 6, 5, 0] 402.94966134575435
CPU times: user 1min, sys: 229 ms, total: 1min
Wall time: 1min 1s


In [9]:
import plotly.graph_objects as go

points = go.Scatter(x=cities_df['x'], y=cities_df['y'], mode='markers',
                    marker=dict(color='#5D69B1', size=10), name='Город',
                    text = ["Название: {}".format(name) for name in cities_df['name'] ])

paths_history, distances_history = tsp.get_solution_in_generations()
generations = len(paths_history)
answer = paths_history[generations - 1]

path = go.Scatter(x=answer[0], y=answer[1], mode='lines', name="Маршрут")

button = {
        "type": "buttons",
        "buttons": [
            {
                "label": "Запустить",
                "method": "animate",
                "args": [None, {"frame": {"duration": 180}}],
            }
        ],
    }

frames = []
for generation in range(generations):
    path = go.Scatter(x=paths_history[generation][0], y=paths_history[generation][1], 
                      mode='lines', name="Маршрут")
    layout = go.Layout(updatemenus=[button], 
                       title_text=f"Маршрут, найденный муравьиным алгоритмом за {generation + 1} шагов, " +
                                  f"длина маршрута = {distances_history[generation]}")
    frame = go.Frame(data=[points, path], layout=layout)
    frames.append(frame)


fig = go.Figure(data=[points, path], frames=frames, layout=layout)
fig.show()